In [15]:
import rospy
from giskardpy.python_interface.python_interface import GiskardWrapper
import numpy as np
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped, PointStamped
from tf.transformations import quaternion_from_matrix, quaternion_about_axis
from giskardpy.goals.adaptive_goals import CloseGripper, PouringAdaptiveTilt
from giskard_msgs.msg import GiskardError

In [2]:
rospy.init_node('giskard_notebook')

The following 4 cells do
1. Restart the simulation. Success is indicated by the reset of the visualization.
2. Start or restart the Giskard node that controls the simulated robot.
3. Starts a script that analyzes the simulation data and publishes bounding boxes for all objects (not for the particles).
4. Starts a script that contains the reasoner that based on the rules and the current context decides the next best motion primitive to influence Giskard. Output can be seen in the topic /reasoner/concluded_behaviour.

Everytime one wants to repeat this experiment, the 4 commands should be executed again. Note that the goal parameter should be set before the reasoner is started.

In [ ]:
%%bash --bg
roslaunch hsr_mujoco hsrb4s_velocity_mixing.launch > / dev / null

In [3]:
%%bash --bg
roslaunch giskardpy giskardpy_hsr_mujoco.launch > / dev / null

In [5]:
giskard = GiskardWrapper()

#### If the robot is not shown but the environment is, then click the reload button in rviz
The robot normally stands to the right of the table.

In [18]:
# giskard.set_avoid_name_conflict(False)
goal_pose = PoseStamped()
goal_pose.header.frame_id = 'map'
goal_pose.pose.orientation = Quaternion(*quaternion_from_matrix([[0, 0, 1, 0],
                                                                 [0, -1, 0, 0],
                                                                 [1, 0, 0, 0],
                                                                 [0, 0, 0, 1]]))
goal_pose.pose.position.x = 1.95
goal_pose.pose.position.y = -0.6
goal_pose.pose.position.z = 0.7
giskard.motion_goals.add_cartesian_pose(goal_pose, 'hand_palm_link', 'map')
giskard.motion_goals.allow_all_collisions()
giskard.add_default_end_motion_conditions()
giskard.execute()

#######
robot_feature = PointStamped()
robot_feature.header.frame_id = 'hand_palm_link'
robot_feature.point = Point(0, 0, 0)

robot_up_axis = Vector3Stamped()
robot_up_axis.header.frame_id = 'hand_palm_link'
robot_up_axis.vector.x = 1

robot_pointing_axis = Vector3Stamped()
robot_pointing_axis.header.frame_id = 'hand_palm_link'
robot_pointing_axis.vector.z = 1

# Define the world features (center and z-axis of the bowl)

world_feature2 = PointStamped()
world_feature2.header.frame_id = 'map'
world_feature2.point = Point(2, -0.2, 0.29)

world_up_axis = Vector3Stamped()
world_up_axis.header.frame_id = 'map'
world_up_axis.vector.z = 1

start_spiral = PointStamped()
start_spiral.header.frame_id = 'map'
start_spiral.point = Point(2, -0.2, 0.45)

seq = [
    [
        ['distance', world_feature2, robot_feature, 0.00, 0.02],
        ['height', world_feature2, robot_feature, 0.25, 0.30],
        ['align', world_up_axis, robot_up_axis]
    ],
    [
        ['height', world_feature2, robot_feature, 0.15, 0.16],
        ['mixing', start_spiral, world_up_axis, 0.05, 25],
        ['angle', world_up_axis, robot_up_axis, 0.0, 0.4],
        ['force', 2, '/mujoco/sensors_3D']
    ],
    [
        ['distance', world_feature2, robot_feature, 0.00, 0.05],
        ['height', world_feature2, robot_feature, 0.25, 0.3],
        ['align', world_up_axis, robot_up_axis]
    ],
]

giskard.create_tcmp_controller(tip_link='hand_palm_link', root_link='map',
                                 sequence=seq, max_vel=0.2)

giskard.monitors.add_max_trajectory_length(60)
result = giskard.execute()
if result.error.code != GiskardError.SUCCESS:
    print(giskard.get_end_motion_reason(move_result=result, show_all=False))

KeyboardInterrupt: 